In [1]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

In [2]:
mol_name = 'H2O'
# mol_name = 'H2'

# mol_name = 'LiH'

In [3]:
xyz_file = xyz_from_pubchem(mol_name)
print(xyz_file)

3
 
O	0	0	0
H	0.2774	0.8929	0.2544
H	0.6068	-0.2383	-0.7169



In [4]:
# xyz_file = '4\n \nH\t0\t0\t0\nH\t1\t0\t0\nH\t2\t0\t0\nH\t3\t0\t0'

In [5]:
from symred.chem import Draw_molecule

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="sphere")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
from symred.chem import PySCFDriver

In [7]:
basis = 'STO-3G'
convergence = 1e6
charge=0
max_hf_cycles=50
ram = 8_000
run_mp2  = True,
run_cisd = True,
run_ccsd = True,
run_fci  = True

In [8]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [9]:
pyscf_obj.run_pyscf()

In [10]:
2*pyscf_obj.pyscf_hf.mol.nao

14

In [11]:
from symred.chem import FermionicHamilt

In [12]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_operator()

H_ferm.fermionic_molecular_hamiltonian

() 9.08436451197603
((0, 1), (0, 0)) -32.68895210663272
((0, 1), (2, 0)) 0.5582454628694448
((0, 1), (4, 0)) 2.54022894090807e-07
((0, 1), (6, 0)) 0.2660800253847062
((0, 1), (10, 0)) 0.31340928606888
((0, 1), (12, 0)) -1.477797333210057e-06
((1, 1), (1, 0)) -32.68895210663272
((1, 1), (3, 0)) 0.5582454628694448
((1, 1), (5, 0)) 2.54022894090807e-07
((1, 1), (7, 0)) 0.2660800253847062
((1, 1), (11, 0)) 0.31340928606888
((1, 1), (13, 0)) -1.477797333210057e-06
((2, 1), (0, 0)) 0.5582454628694435
((2, 1), (2, 0)) -7.581925631735151
((2, 1), (4, 0)) 3.787547667638218e-08
((2, 1), (6, 0)) -0.488495142106376
((2, 1), (10, 0)) -1.4460616540254612
((2, 1), (12, 0)) 7.007873807317976e-06
((3, 1), (1, 0)) 0.5582454628694435
((3, 1), (3, 0)) -7.581925631735151
((3, 1), (5, 0)) 3.787547667638218e-08
((3, 1), (7, 0)) -0.488495142106376
((3, 1), (11, 0)) -1.4460616540254612
((3, 1), (13, 0)) 7.007873807317976e-06
((4, 1), (0, 0)) 2.5402289396460144e-07
((4, 1), (2, 0)) 3.787547678669034e-08
((4, 1)

In [13]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [14]:
hf_ket = H_ferm.hf_ket
hf_ket.shape

(16384,)

In [15]:
## check
from functools import reduce
state = {0: np.array([[1],[0]]), 1: np.array([[0],[1]])}
ll = [state[bit] for bit in hf_state]
kket = reduce(np.kron, ll)
all(kket[:,0] == hf_ket)

True

In [16]:
print(H_ferm.scf_method.energy_tot())
print(pyscf_obj.pyscf_hf.energy_tot())

H_mat = H_ferm.get_sparse_ham()
hf_ket.conj().T @ H_mat @ hf_ket

-74.96386325726523
-74.96386325726523


(-74.96386325726526+0j)

In [17]:
# def compute_mp2_amplitudes(self) -> ClosedShellAmplitudes:
#     """
#     Compute closed-shell mp2 amplitudes
#     .. math::
#         t(a,i,b,j) = 0.25 * g(a,i,b,j)/(e(i) + e(j) -a(i) - b(j) )
#     :return:
#     Parameters
#     ----------
#     Returns
#     -------
#     """
#     g = self.molecule.two_body_integrals
#     fij = self.molecule.orbital_energies
#     nocc = self.molecule.n_electrons // 2  # this is never the active space
#     ei = fij[:nocc]
#     ai = fij[nocc:]
#     abgij = g[nocc:, nocc:, :nocc, :nocc]
#     amplitudes = abgij * 1.0 / (
#             ei.reshape(1, 1, -1, 1) + ei.reshape(1, 1, 1, -1) - ai.reshape(-1, 1, 1, 1) - ai.reshape(1, -1, 1, 1))
#     E = 2.0 * numpy.einsum('abij,abij->', amplitudes, abgij) - numpy.einsum('abji,abij', amplitudes, abgij,
#                                                                             optimize='greedy')

#     self.molecule.mp2_energy = E + self.molecule.hf_energy
#     return ClosedShellAmplitudes(tIjAb=numpy.einsum('abij -> ijab', amplitudes, optimize='greedy'))

In [39]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
ei = fij[:nocc]
ai = fij[nocc:]
abgij = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (
        ei.reshape(1, 1, -1, 1) + ei.reshape(1, 1, 1, -1) - ai.reshape(-1, 1, 1, 1) - ai.reshape(1, -1, 1, 1))
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

-0.03732165013343573

In [ ]:
pyscf_obj.pyscf_mp2.e_corr

In [40]:
tIjAb=np.einsum('abij -> ijab', amplitudes, optimize='greedy')

tIjAb.shape

(5, 5, 2, 2)

In [ ]:
# tIjAb=np.einsum('abij -> abij', amplitudes, optimize='greedy')

# tIjAb.shape

In [ ]:
# pyscf_obj.pyscf_mp2.t2 # <-- these are the amplitudes!

In [ ]:
# variables = {}
# if tIjAb is not None:
#     nvirt = tIjAb.shape[2]
#     nocc = tIjAb.shape[0]
#     assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
#     for (I, J, A, B), value in np.ndenumerate(tIjAb):
#         if not np.isclose(value, 0.0, atol=1e-8):
#             variables[(nocc + A, I, nocc + B, J)] = value
            

nvirt = tIjAb.shape[2]
nocc = tIjAb.shape[0]
excite_op =FermionOperator()
assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
for (I, J, A, B), value in np.ndenumerate(tIjAb):
    if not np.isclose(value, 0.0, atol=1e-8):
        excite_op+= FermionOperator(f'{nocc + A}^ {I}^ {nocc + B} {J}', value)
#         excite_op+= FermionOperator(f'{I}^ {J}^ {nocc + A} {nocc + B}', value)
#         excite_op+= FermionOperator(f'{nocc + A}^ {nocc + B}^ {I} {J}', -value)

In [ ]:
tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

nvirt = tijab.shape[2]
nocc = tijab.shape[0]
excite_op =FermionOperator()
assert (tijab.shape[1] == nocc and tijab.shape[3] == nvirt)
for (I, J, A, B), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=1e-8):
        excite_op+= FermionOperator(f'{nocc + A}^ {I}^ {nocc + B} {J}', value)

In [57]:
tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

tIjAb=tIjAb
tiJaB=tIjAb
tijab=tijab
tIJAB=tijab


threshold = 1e-8
excite_op =FermionOperator()
nvirt = tIjAb.shape[2]
nocc = tIjAb.shape[0]
assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)

for (I, j, A, b), value in np.ndenumerate(tIjAb):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + A)}^ {2*I}^ {2 * (nocc + b) + 1} {j + 1}', value/2)
#         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + b) + 1, j + 1)] = value
for (i, J, a, B), value in np.ndenumerate(tiJaB):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + a)+1}^ {2*i+1}^ {2 * (nocc + B)} {J}', value/2)
#         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + B), J)] = value
for (i, j, a, b), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=threshold):
        excite_op+= FermionOperator(f'{2 * (nocc + a)+1}^ {2*i+1}^ {2 * (nocc + b)+1} {j+1}', value/2)
#         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + b) + 1, j + 1)] = value
for (I, J, A, B), value in np.ndenumerate(tijab):
    if not np.isclose(value, 0.0, atol=threshold):
#         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + B), J)] = value
        excite_op+= FermionOperator(f'{2 * (nocc + A)}^ {2*I}^ {2 * (nocc + B)} {J}', value/2)

In [62]:
# tijab = tIjAb - np.einsum("ijab -> ijba", tIjAb, optimize='greedy')

# tIjAb=tIjAb
# tiJaB=tIjAb
# tijab=tijab
# tIJAB=tijab


# threshold = 1e-8
# excite_op =FermionOperator()
# nvirt = tIjAb.shape[2]
# nocc = tIjAb.shape[0]
# assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)

# for (I, j, A, b), value in np.ndenumerate(tIjAb):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + A), 2*I, 2 * (nocc + b) + 1, j + 1]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + A), 2 * I, 2 * (nocc + b) + 1, j + 1)] = value
# for (i, J, a, B), value in np.ndenumerate(tiJaB):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + a)+1, 2*i+1, 2 * (nocc + B), J]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + B), J)] = value
# for (i, j, a, b), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + a)+1, 2*i+1, 2 * (nocc + b)+1, j+1]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)
# #         variables[(2 * (nocc + a) + 1, 2 * i + 1, 2 * (nocc + b) + 1, j + 1)] = value
# for (I, J, A, B), value in np.ndenumerate(tijab):
#     if not np.isclose(value, 0.0, atol=threshold):
#         term = [2 * (nocc + A), 2*I, 2 * (nocc + B), J]
#         excite_op+= FermionOperator(f'{term[0]}^ {term[1]}^ {term[2]} {term[3]}', value/2)
#         excite_op+= FermionOperator(f'{2*term[0]+1}^ {2*term[1]+1}^ {2*term[2]+1} {2*term[3]+1}', value/2)

In [61]:
len(list(excite_op))

# excite_op/2

264

In [47]:
len(list(generator))

240

In [42]:
generator

-0.0003873711400727807 [10^ 0 11^ 1] +
-0.000155447177116903 [10^ 0 11^ 3] +
-1.515964749202942e-05 [10^ 0 11^ 7] +
0.00025334423673538343 [10^ 0 12^ 4] +
0.00016040797448131825 [10^ 0 13^ 5] +
0.0003873711400727807 [10^ 1 11^ 0] +
0.0001554471771169041 [10^ 1 11^ 2] +
1.515964749203047e-05 [10^ 1 11^ 6] +
9.293626225406507e-05 [10^ 1 13^ 4] +
-0.0001554471771169041 [10^ 2 11^ 1] +
-0.014907303810801466 [10^ 2 11^ 3] +
-8.639519514653979e-08 [10^ 2 11^ 5] +
0.010749432877645813 [10^ 2 11^ 7] +
-0.006772348763958736 [10^ 2 12^ 4] +
3.215691464355366e-08 [10^ 2 13^ 3] +
-0.012848728802469686 [10^ 2 13^ 5] +
-5.568183723932386e-08 [10^ 2 13^ 7] +
0.000155447177116903 [10^ 3 11^ 0] +
0.014907303810801466 [10^ 3 11^ 2] +
8.639519514538644e-08 [10^ 3 11^ 4] +
-0.010749432877645801 [10^ 3 11^ 6] +
-3.215691464355366e-08 [10^ 3 13^ 2] +
0.006076380038510953 [10^ 3 13^ 4] +
5.9323920089013677e-08 [10^ 3 13^ 6] +
-8.639519514538644e-08 [10^ 4 11^ 3] +
-0.014637105256511138 [10^ 4 11^ 5] +
1.2653

In [ ]:
mp2_ansatz = expm(get_sparse_operator(0.25*excite_op, n_qubits=H_ferm.n_qubits))
## mp2_ansatz = expm(get_sparse_operator(excite_op, n_qubits=H_ferm.n_qubits))

## help(qubit_operator_sparse)

In [ ]:
mp2_ket =  mp2_ansatz @ hf_ket
mp2_ket.conj().T @ H_mat @ mp2_ket

In [ ]:
pyscf_obj.pyscf_mp2.e_tot

In [ ]:
pyscf_obj.pyscf_mp2.e_hf

In [ ]:
hf_ket.conj().T @ mp2_ket

In [ ]:
# t(a,i,b,j) = 0.25 * g(a,i,b,j)/(e(i) + e(j) -a(i) - b(j) )

# +aibj
# -abij

In [ ]:
pyscf_obj.pyscf_mp2.e_corr #+ pyscf_obj.pyscf_mp2.e_corr

In [ ]:
variables = {}
if tIjAb is not None:
    nvirt = tIjAb.shape[2]
    nocc = tIjAb.shape[0]
    assert (tIjAb.shape[1] == nocc and tIjAb.shape[3] == nvirt)
    for (I, J, A, B), value in np.ndenumerate(tIjAb):
        if not np.isclose(value, 0.0, atol=1e-8):
            variables[(nocc + A, I, nocc + B, J)] = value

dict(sorted(variables.items(), key=lambda x: np.abs(x[1]), reverse=True))

In [20]:
from pyscf.cc.addons import spatial2spin

t2 = spatial2spin(pyscf_obj.pyscf_mp2.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

In [33]:
# double_amplitudes_list=[]
# double_amplitudes = double_amps
# for i, j, k, l in zip(*double_amplitudes.nonzero()):
#     double_amplitudes_list.append([[i, j, k, l],
#                                    double_amplitudes[i, j, k, l]])
    
# double_amplitudes_list

double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])
    
double_amplitudes_list

[[[10, 0, 11, 1], -0.0003873711400727807],
 [[10, 0, 11, 3], -0.000155447177116903],
 [[10, 0, 11, 7], -1.515964749202942e-05],
 [[10, 0, 12, 4], 0.00025334423673538343],
 [[10, 0, 13, 5], 0.00016040797448131825],
 [[10, 1, 11, 0], 0.0003873711400727807],
 [[10, 1, 11, 2], 0.0001554471771169041],
 [[10, 1, 11, 6], 1.515964749203047e-05],
 [[10, 1, 13, 4], 9.293626225406507e-05],
 [[10, 2, 11, 1], -0.0001554471771169041],
 [[10, 2, 11, 3], -0.014907303810801466],
 [[10, 2, 11, 5], -8.639519514653979e-08],
 [[10, 2, 11, 7], 0.010749432877645813],
 [[10, 2, 12, 4], -0.006772348763958736],
 [[10, 2, 13, 3], 3.215691464355366e-08],
 [[10, 2, 13, 5], -0.012848728802469686],
 [[10, 2, 13, 7], -5.568183723932386e-08],
 [[10, 3, 11, 0], 0.000155447177116903],
 [[10, 3, 11, 2], 0.014907303810801466],
 [[10, 3, 11, 4], 8.639519514538644e-08],
 [[10, 3, 11, 6], -0.010749432877645801],
 [[10, 3, 13, 2], -3.215691464355366e-08],
 [[10, 3, 13, 4], 0.006076380038510953],
 [[10, 3, 13, 6], 5.9323920089

In [34]:
generator = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [35]:
mp2_ansatz = expm(get_sparse_operator(0.25*generator, n_qubits=H_ferm.n_qubits))
# mp2_ansatz = expm(get_sparse_operator(0.125*generator, n_qubits=H_ferm.n_qubits))

In [36]:
mp2_ket =  mp2_ansatz @ hf_ket
mp2_ket.conj().T @ H_mat @ mp2_ket

(-75.26246933247938+0j)

In [37]:
pyscf_obj.pyscf_mp2.e_tot

-75.00118490739867

In [38]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
ei = fij[:nocc]
ai = fij[nocc:]
V = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (
        ei.reshape(1, 1, -1, 1) + ei.reshape(1, 1, 1, -1) - ai.reshape(-1, 1, 1, 1) - ai.reshape(1, -1, 1, 1))
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

-0.03732165013343573

https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.154.2423&rep=rep1&type=pdf

In [27]:
mp2_ansatz = expm(get_sparse_operator(0.5*generator, n_qubits=H_ferm.n_qubits))
# mp2_ansatz = expm(get_sparse_operator(0.125*generator, n_qubits=H_ferm.n_qubits))

In [28]:
mp2_ket =  mp2_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ mp2_ket

(-75.00118490739868+0j)

In [29]:
pyscf_obj.pyscf_mp2.e_corr

-0.037321650133435705

In [32]:
(hf_ket.conj().T @ H_mat @ mp2_ket - pyscf_obj.pyscf_mp2.e_hf).real

-0.03732165013344968